In [ ]:
from sklearn import decomposition
from sklearn.metrics import confusion_matrix
from collections import Counter
import heapq
import numpy as np
import math

In [1]:
class Node():
    def __init__(self, isleaf, xmin, xmax, ymin, ymax, xmid = None, ymid = None, coord = None, classifier = None, ul = None, ur = None, ll = None, lr = None, _parent = None):
        self.isleaf = isleaf # True / False 
        self.coord = coord 
        self.classifier = classifier 
        self.nodecount = 0 # node numbers
        # set the boundary of the node
        self.xmin = xmin
        self.xmid = xmid # median     
        self.xmax = xmax
        self.ymin = ymin
        self.ymid = ymid # median      
        self.ymax = ymax
        # subnode
        self.ul = ul
        self.ur = ur
        self.ll = ll
        self.lr = lr
        self._parent = _parent


In [2]:
class QdTree():
    def __init__(self, xmin, xmax, ymin, ymax):
        self.root = Node(True, xmin, xmax, ymin, ymax)

    # insert 1 node
    def insert_node(self, node, data):
        if len(data)==0:
            return

        elif len(data)==1:
            node.nodecount = 1
            node.isleaf = True
            node.coord = [data["PC0"].iloc[0],data["PC1"].iloc[0]]
            node.classifier=data["CLASS"].iloc[0]
        
        else:
            node.isleaf = False
            node.nodecount = len(data) 
            
            node.xmid = np.median(data["PC0"])
            node.ymid = np.median(data["PC1"])

            ul_data = data[(data["PC0"] < node.xmid) and (data["PC1"] >= node.ymid)]        
            ur_data = data[(data["PC0"] >= node.xmid) and (data["PC1"]>=node.ymid)]
            ll_data = data[(data["PC0"]<node.xmid) and (data["PC1"]<node.ymid)]
            lr_data = data[(data["PC0"]>=node.xmid) and (data["PC1"]<node.ymid)]
            node.ul = Node(1,node.xmin,node.xmid,node.ymid,node.ymax,_parent=node)
            self.insert(node.ul,ul_data)          
            node.ur = Node(1,node.xmid,node.xmax,node.ymid,node.ymax,_parent=node)
            self.insert(node.ur,ur_data)                                
            node.ll = Node(1,node.xmin,node.xmid,node.ymin,node.ymid,_parent=node)            
            self.insert(node.ll,ll_data)
            node.lr = Node(1,node.xmid,node.xmax,node.ymin,node.ymid,_parent=node) 
            self.insert(node.lr, lr_data)
            if not node.ur.nodecount:
                node.ur = None
            if not node.ul.nodecount:
                node.ul = None 
            if not node.ll.nodecount:
                node.ll = None 
            if not node.lr.nodecount:
                node.lr = None 


    def search_node(self, coord, count = 1):
        root = self.root
        parent = None
        while root and not root.isleaf:
            if root.nodecount <= count:
                return root

            if coord[0] < root.xmid and coord[1] < root.ymid:
                parent = root
                root = root.ll

            elif coord[0] < root.xmid and coord[1] >= root.ymid:
                parent = root
                root = root.ul

            elif coord[0] >= root.xmid and coord[1] < root.ymid:
                parent = root
                root = root.lr

            else:
                parent = root
                root = root.ur

        return root if root else parent
    
    def euclidean_distance(self, x, y):
        dist = (x[0] - y[0]) ** 2 + (x[1] - y[1]) ** 2
        return math.sqrt(dist)

    def _within_distance(self,coord,node,d):
        if coord[0]<node.xmin and coord[1]<node.ymin:
            return self.euclidean_distance(coord,(node.xmin,node.ymin))<=d
        elif coord[0]<node.xmin and coord[1]<node.ymax:
            return self.euclidean_distance(coord,(node.xmin,coord[1]))<=d
        elif coord[0]<node.xmin and coord[1]>=node.ymax:
            return self.euclidean_distance(coord,(node.xmin,node.ymax))<=d
        elif coord[0]<node.xmax and coord[1]<node.ymin:
            return self.euclidean_distance(coord,(coord[0],node.ymin))<=d
        elif coord[0]<node.xmax and coord[1]<node.ymax:
            return True
        elif coord[0]<node.xmax and coord[1]>=node.ymax:
            return self.euclidean_distance(coord,(coord[0],node.ymax))<=d          
        elif coord[0]>=node.xmax and coord[1]<node.ymin:
            return self.euclidean_distance(coord,(node.xmax,node.ymin))<=d
        elif coord[0]>=node.xmax and coord[1]<node.ymax:
            return self.euclidean_distance(coord,(node.xmax,node.ymax))<=d
        elif coord[0]>=node.xmax and coord[1]>=node.ymax:
            return self.euclidean_distance(coord,(node.xmax,node.ymax))<=d

    def get_neighbors(self, coord, k):
        neighbors = []
        lst = [self.search_node(coord,count=k)]

        while lst:
            item=lst.pop(0) 
            if item.leaf:
                euclidean_distance = self.euclidean_distance(item.coord, coord)
                heapq.heappush(neighbors,(-euclidean_distance, item))
            else:
                for child in [item.ll,item.lr,item.ul,item.ur]:
                    if child:
                        lst.append(child)                  
        lst=[self.root]
        while lst:
            item=lst.pop(0)
            if item.leaf:
                euclidean_distance=self.euclidean_distance(item.coord,coord)
                heapq.heappush(neighbors,(-euclidean_distance, item))
                if len(neighbors) > k:
                    heapq.heappop(neighbors)
            else:
                for child in [item.ll,item.lr,item.ul,item.ur]: 
                    if child and child != (self.search_node(coord,count=k)):
                        if len(neighbors)<k or self._within_distance(coord,child,-neighbors[0][0]):
                            lst.append(child)
        return neighbors

In [ ]:
datatree=QdTree(min(train["PC0"]),max(train["PC0"]),min(train["PC1"]),max(train["PC1"]))
datatree.insert(datatree.root,train)
for a,b in test.iterrows():
    res = datatree.findneighbors([b["PC0"],b["PC1"]],k)
    count_list = []
    for p in res:
        count_list.append(p[1].class_)
    res_qdtree = Counter(count_list).most_common(1)[0][0]  
    quad_result.append(res_qdtree)
true = true + test['CLASS'].to_list()
print(confusion_matrix(quad_result,true)) 

In [ ]:
datatree=QdTree(min(train["PC0"]),max(train["PC0"]),min(train["PC1"]),max(train["PC1"]))
datatree.insert(datatree.root,train)
for a,b in test.iterrows():
    res = datatree.findneighbors([b["PC0"],b["PC1"]],k)
    count_list = []
    for p in res:
        count_list.append(p[1].class_)
    res_qdtree = Counter(count_list).most_common(1)[0][0]  
    quad_result.append(res_qdtree)
true = true + test['CLASS'].to_list()
print(confusion_matrix(quad_result,true)) 